In [1]:
from CNOModule import CNO
import torch
from torch.utils.data import Dataset, DataLoader
from scipy.io import loadmat

/ext3/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.cuda.is_available():
    # Set the default device to CUDA
    device = torch.device('cuda')
    #torch.set_default_device(device)
    print('Using CUDA for tensor operations')
    torch.cuda.empty_cache()
else:
    device = torch.device('cpu')
    print('CUDA is not available. Using CPU for tensor operations')

Using CUDA for tensor operations


In [3]:
class CustomDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data
        
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x, y

In [4]:
NavierStokes_mat = loadmat('../NavierStokes_V1e-5_N1200_T20.mat')

In [5]:
# Assuming NavierStokes_mat['u'] and NavierStokes_mat['a'] are numpy arrays
u_numpy = NavierStokes_mat['u']
a_numpy = NavierStokes_mat['a']

# Convert numpy arrays to PyTorch tensors
u_tensor = torch.tensor(u_numpy)
a_tensor = torch.tensor(a_numpy)

# Concatenate along the desired dimension
concatenated_tensor = torch.cat((u_tensor[:,:,:,:10], a_tensor.unsqueeze(-1)), dim=-1).permute(0, 3, 1, 2).to(device)
result_tensor = u_tensor[:,:,:,:10].permute(0, 3, 1, 2).to(device)

In [6]:
# Assuming you have concatenated_tensor as x_data and result_tensor as y_data
my_dataset = CustomDataset(concatenated_tensor, result_tensor)

In [7]:
# Define your DataLoader
batch_size = 64  # Adjust batch size as needed
train_loader = DataLoader(my_dataset, batch_size=batch_size, shuffle=True)

# CNO

In [ ]:
# Define your training parameters
learning_rate = 0.001
num_epochs = 2000  # Adjust as needed

#train_loader = data_loader
#test_loader = data_loader

# Initialize your model
model = CNO(in_dim  = 11,
            out_dim = 10, # Number of input channels.
            N_layers = 5,
            in_size = 64,
            out_size = 64).to(device).to(device)

# Define loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    # Training
    model.train()
    running_loss = 0.0

    # Iterate over batches
    for batch_data, batch_targets in train_loader:
        # Move data to device
        batch_data, batch_targets = batch_data.to(device), batch_targets.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_data)

        # Compute the loss
        loss = criterion(outputs, batch_targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate average training loss for the epoch
    train_loss = running_loss / len(train_loader)

    if (epoch + 1) % 10 == 0:
        # Print the loss for each epoch
        print(f'Epoch [{epoch + 1}/{num_epochs}], '
              f'Training Loss: {train_loss:.8e} ')

print('Finished Training')

Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.
Epoch [10/2000], Training Loss: 5.27815131e-03 
Epoch [20/2000], Training Loss: 3.65142100e-03 
Epoch [30/2000], Training Loss: 2.63211034e-03 
